In [ ]:
! pip install git+https://github.com/CUAHSI/argo-workflow-python-client.git

In [ ]:
import swagger_client

# Configure API key authorization: BearerToken
configuration = swagger_client.Configuration()
configuration.api_key['Authorization'] = '<BearerToken>'
configuration.host = "https://workflows.argo.cuahsi.io"
template_api_instance = swagger_client.WorkflowTemplateServiceApi(swagger_client.ApiClient(configuration))
workflow_api_instance = swagger_client.WorkflowServiceApi(swagger_client.ApiClient(configuration))

namespace = 'workflows'

wb_id = "wb-2851653"
vpu_id = "16"
output_bucket = "sblack"
start_date = "2020-01-01"
end_date = "2020-01-02"
hydrofabric_url = "s3://lynker-spatial/hydrofabric/v20.1/gpkg/"
output_path = "{{workflow.labels.workflows.argoproj.io/creator-preferred-username}}" + f"/{wb_id}"


In [ ]:
def submit_workflow_body(template_name: str, parameters: dict):
    parameters_list = [f"{key}={value}" for key, value in parameters.items()]
    return {
        "resourceKind": "WorkflowTemplate",
        "resourceName": template_name,
        "submitOptions": {
            "parameters":parameters_list,
        }
    }

In [ ]:
def wait_for_completion(workflow_id):
    import time
    my_submitted_workflow = workflow_api_instance.workflow_service_get_workflow(namespace, workflow_id)
    phase = my_submitted_workflow.status.phase
    print(phase)
    while my_submitted_workflow.status.phase == 'Running':
        my_submitted_workflow = workflow_api_instance.workflow_service_get_workflow(namespace, workflow_id)
        phase = my_submitted_workflow.status.phase
        print(phase)
        time.sleep(5)

In [ ]:

parameters = {
    "wb-id": wb_id,
    "vpu-id": vpu_id,
    "output-bucket": output_bucket,
    "hydrofabric-url": hydrofabric_url,
    "output-path": output_path,
}
body = submit_workflow_body("ngen-v2.0-subset-minio", parameters)
api_response = workflow_api_instance.workflow_service_submit_workflow(namespace, body)
wait_for_completion(api_response.metadata.name)

In [ ]:
parameters = {
    "shape-file-path": f"{output_path}/domain/watershed.gz",
    "input-bucket": output_bucket,
    "output-bucket": output_bucket,
    "output-path": f"{output_path}/forcing",
    "start-date": start_date,
    "end-date": end_date,
    #"aorc-secret": "secret secret, I got a secret"
}
body = submit_workflow_body("collect-aorc-forcing-v1.1", parameters)
api_response = workflow_api_instance.workflow_service_submit_workflow(namespace, body)
wait_for_completion(api_response.metadata.name)

In [ ]:
parameters = {
    "forcing-bucket": output_bucket,
    "forcing-path": f"{output_path}/forcing",
    "forcing-file-name": "forcing.nc",
    "domain-bucket": output_bucket,
    "domain-path": f"{output_path}/domain",
    "geopackage-file-name": f"{wb_id}_upstream_subset.gpkg",
    "noaaowp-attributes-file-name": "cfe_noahowp_attributes.csv",
    "output-bucket": output_bucket,
    "output-path": output_path,
}
body = submit_workflow_body("cfe-ngen-configure", parameters)
api_response = workflow_api_instance.workflow_service_submit_workflow(namespace, body)
wait_for_completion(api_response.metadata.name)

In [ ]:
parameters = {
    "input-data-bucket": output_bucket,
    "input-data-path": output_path,
    "catchment-file-path": "domain/catchments.geojson",
    "nexus-file-path": "domain/nexus.geojson",
    "realization-file-path": "config/realization.json",
    "output-bucket": output_bucket,
    "output-path": output_path,
}
body = submit_workflow_body("ngen-run", parameters)
api_response = workflow_api_instance.workflow_service_submit_workflow(namespace, body)
wait_for_completion(api_response.metadata.name)